In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import re
import sqlparse
from os.path import abspath
import pyspark.sql.functions as f
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark import SparkContext, SparkConf
import os
import pandas as pd
import warnings
import findspark
import tensorflow as tf
import keras
from keras import layers
from keras.layers import Input, Embedding, Dot, Reshape, Dense, Add, Activation
from keras.models import Model
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.functions import expr
from pyspark.sql.functions import when, col

In [ ]:
findspark.init("C:\recomendation_systems\spark")
warnings.filterwarnings('ignore')

warehouse_location = abspath('spark-warehouse')


os.environ["SPARK_HOME"] = r"C:\recomendation_systems\spark"
os.environ["JAVA_HOME"] = r"C:\Program Files\Java\jdk-20"
os.environ["HADOOP_HOMe"]=r"C:\recomendation_systems\spark\hadoop\bin"

In [ ]:
conf = SparkConf()
conf.set("spark.master","local[*]")
conf.set('spark.driver.extraClassPath', r"C:\recomendation_systems\sqljdbc_12.4")
conf.set('spark.executor.extraClassPath', r"C:\recomendation_systems\sqljdbc_12.4")
spark = SparkSession.builder\
        .config(conf=conf)\
        .config("spark.sql.warehouse.dir", warehouse_location)\
        .getOrCreate()

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:


spark = SparkSession.builder.appName("app").getOrCreate()


data = [Row(nome="valor1", idade=25, cidade="São Paulo"),
        Row(nome="valor2", idade=30, cidade="Rio de Janeiro"),
        Row(nome="valor3", idade=22, cidade="São Paulo"),
        Row(nome="valor4", idade=35, cidade="Belo Horizonte")]

dataframe = spark.createDataFrame(data)


def identificar_tipo_consulta(consulta):
    if consulta.strip().startswith("SELECT"):
        return "sql"
    elif consulta.strip().startswith("spark.sql"):
        return "pyspark"
    else:
        raise ValueError("Tipo de consulta não suportado")


def extrair_campos_sql(consulta):
    parsed = sqlparse.parse(consulta)
    if parsed:
        first_statement = parsed[0]
        fields = []
        for token in first_statement.tokens:
            if isinstance(token, sqlparse.sql.IdentifierList):
                for identifier in token.get_identifiers():
                    fields.append(identifier.value)
            elif isinstance(token, sqlparse.sql.Identifier):
                fields.append(token.value)
        return fields
    else:
        return []


def gerar_consulta_pyspark(campos_sql):
    modelo_pyspark = "df.select({})".format(", ".join(campos_sql))
    return modelo_pyspark


def executar_consulta_pyspark(consulta_pyspark, dataframe):
    resultado = dataframe.sql(consulta_pyspark)
    return resultado


def consulta_decorator(func):
    def wrapper(consulta, dataframe):
        tipo_consulta = identificar_tipo_consulta(consulta)
        if tipo_consulta == 'sql':
            campos_sql = extrair_campos_sql(consulta)
            consulta_pyspark = gerar_consulta_pyspark(campos_sql)
            resultado = executar_consulta_pyspark(consulta_pyspark, dataframe)
            return resultado
        elif tipo_consulta == 'pyspark':
            return func(consulta, dataframe)
        else:
            raise ValueError("Tipo de consulta não suportado")
    return wrapper

@consulta_decorator
def executar_consulta(consulta, dataframe):
    return dataframe.sql(consulta)


consulta_sql = "SELECT nome, idade FROM clientes WHERE cidade = 'São Paulo'"

resultado = executar_consulta(consulta_sql, dataframe)

resultado.show()

In [21]:
df = pd.read_csv(r"C:\recomendation_systems\querys_treino.csv", sep=";", encoding="latin-1") 

In [22]:
df.columns

Index(['codigo', 'tipo', 'funcao'], dtype='object')

,codigo
0,"df.select(""firstname"",""lastname"").show()"
1,"df.select(df.firstname,df.lastname).show()"
2,"df.select(df[""firstname""],df[""lastname""]).show()"
3,from pyspark.sql.functions import col
4,"df.select(col(""firstname""),col(""lastname"")).sh..."


In [30]:
import pandas as pd
import nltk
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


nltk.download('stopwords')
nltk.download('wordnet')


base_treino = pd.read_csv("querys_treino.csv", sep=";", encoding="latin-1", usecols=['codigo', 'tipo', 'funcao'])
base_avaliacao = pd.read_csv("avaliaselectspark.csv", sep=";", encoding="latin-1", usecols=['codigo'])


spark_keywords = ['spark.sql', 'delta', 'df.select', 'join', 'df', 'pd']
sql_keywords = ['select', 'update', 'delete', 'merge', '%sql']


X_train = base_treino['codigo']
y_train = base_treino['tipo']

X_test = base_avaliacao['codigo']


def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word.lower() not in stop_words]

    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    processed_text = ' '.join(words)

    processed_text = re.sub(r'([.,!?()])', r' \1 ', processed_text)

    return processed_text


X_train = X_train.apply(preprocess_text)
X_test = X_test.apply(preprocess_text)


vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = vectorizer.transform(X_test).toarray()


def add_keywords_features(text, keywords):
    features = [1 if keyword in text else 0 for keyword in keywords]
    return features

X_train_keywords = X_train.apply(lambda x: add_keywords_features(x, spark_keywords + sql_keywords))
X_test_keywords = X_test.apply(lambda x: add_keywords_features(x, spark_keywords + sql_keywords))

X_train_tfidf = np.hstack((X_train_tfidf, np.array(X_train_keywords.tolist())))
X_test_tfidf = np.hstack((X_test_tfidf, np.array(X_test_keywords.tolist())))


label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)


def build_model(input_shape, num_classes):
    model = Sequential()
    model.add(Dense(256, input_shape=input_shape, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


model = build_model((X_train_tfidf.shape[1],), len(label_encoder.classes_))


model.fit(X_train_tfidf, y_train_encoded, epochs=2000, batch_size=128, verbose=2)


predictions = model.predict(X_test_tfidf)


y_test_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))


base_avaliacao['tipo_previsto'] = y_test_labels


print("Resultados:")
print(base_avaliacao[['codigo', 'tipo_previsto']])



Epoch 1/2000


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


1/1 - 1s - loss: 0.6932 - accuracy: 0.4186 - 721ms/epoch - 721ms/step
Epoch 2/2000
1/1 - 0s - loss: 0.6828 - accuracy: 0.6512 - 4ms/epoch - 4ms/step
Epoch 3/2000
1/1 - 0s - loss: 0.6834 - accuracy: 0.6047 - 3ms/epoch - 3ms/step
Epoch 4/2000
1/1 - 0s - loss: 0.6728 - accuracy: 0.6279 - 0s/epoch - 0s/step
Epoch 5/2000
1/1 - 0s - loss: 0.6810 - accuracy: 0.6047 - 0s/epoch - 0s/step
Epoch 6/2000
1/1 - 0s - loss: 0.6647 - accuracy: 0.6279 - 0s/epoch - 0s/step
Epoch 7/2000
1/1 - 0s - loss: 0.6501 - accuracy: 0.7209 - 5ms/epoch - 5ms/step
Epoch 8/2000
1/1 - 0s - loss: 0.6589 - accuracy: 0.6047 - 3ms/epoch - 3ms/step
Epoch 9/2000
1/1 - 0s - loss: 0.6549 - accuracy: 0.6512 - 8ms/epoch - 8ms/step
Epoch 10/2000
1/1 - 0s - loss: 0.6538 - accuracy: 0.6744 - 4ms/epoch - 4ms/step
Epoch 11/2000
1/1 - 0s - loss: 0.6432 - accuracy: 0.6512 - 1ms/epoch - 1ms/step
Epoch 12/2000
1/1 - 0s - loss: 0.5935 - accuracy: 0.7674 - 9ms/epoch - 9ms/step
Epoch 13/2000
1/1 - 0s - loss: 0.6214 - accuracy: 0.6744 - 4ms/e